<a href="https://colab.research.google.com/github/vinayhpandya/CS298_Project/blob/main/Trying_alberta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!ls

__MACOSX  malware_complete.csv	malware_complete.csv.zip  sample_data


In [ ]:
!unzip malware_completne.csv.zip

Archive:  malware_complete.csv.zip
replace malware_complete.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
import pandas as pd
df=pd.read_csv("malware_complete.csv")

In [ ]:
df.isnull().values.any()

True

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

(5807, 3)

In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 15.0 MB/s 
     |████████████████████████████████| 895 kB 69.0 MB/s 
     |████████████████████████████████| 77 kB 8.5 MB/s 
     |████████████████████████████████| 6.6 MB 71.6 MB/s 
     |████████████████████████████████| 596 kB 77.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

# specify GPU
device = torch.device("cuda")

In [ ]:
from transformers import AlbertModel,AlbertTokenizerFast

In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Sun Apr 24 03:22:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    25W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df.head()

,Unnamed: 0,binaries,label
0,0,push mov cmp jne mov leave ret cmp jne cmp je ...,0
1,1,mov ret int3 int3 int3 int3 int3 int3 int3 int...,0
2,2,mov mov push push push push mov push mov push ...,0
3,3,mov ret int3 int3 int3 int3 int3 int3 int3 int...,0
4,4,mov mov push push push push mov push mov push ...,0


In [ ]:
train_text, temp_text, train_labels, temp_labels = train_test_split(df['binaries'], df['label'], 
                                                                    random_state=42, 
                                                                    test_size=0.3, 
                                                                    stratify=df['label'])


val_text, test_text, val_labels, test_labels = train_test_split(temp_text, temp_labels, 
                                                                random_state=42, 
                                                                test_size=0.5, 
                                                                stratify=temp_labels)

In [ ]:
from transformers import RobertaModel,RobertaTokenizerFast

In [ ]:
#bert = AutoModel.from_pretrained('bert-base-uncased')
# Load the BERT tokenizer
tokenizer = AlbertTokenizerFast.from_pretrained('albert-base-v2')

In [ ]:
del bert

In [ ]:
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),truncation=True,
    padding="max_length",max_length=400,
    return_attention_mask=True,return_tensors="pt"
)

# tokenize and encode sequences in the validation set
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(),truncation=True,
    padding="max_length",max_length=400,
    return_attention_mask=True,return_tensors="pt"
)

In [ ]:
del tokens_val

In [ ]:
del tokens_train

In [ ]:
#train_seq = torch.tensor(tokens_train['input_ids'])
#train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

#val_seq = torch.tensor(tokens_val['input_ids'])
#val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

In [ ]:
from torch.utils.data import Dataset

In [ ]:
from os import truncate
class Malware_Dataset(Dataset):
  def __init__(self,tokenizer,df_binaries,df_labels):
    self.tokenizer=tokenizer
    self.df_binaries=df_binaries
    self.df_labels=df_labels

  def __len__(self):
    return len(self.df_binaries)
  def __getitem__(self, index):
    binaries=self.df_binaries.iloc[index]
    labels=self.df_labels.iloc[index]
    binaries=self.tokenizer.encode_plus(binaries,add_special_tokens=True,
      max_length=400,
      return_token_type_ids=False,
      pad_to_max_length=True,
      return_attention_mask=True,
      return_tensors='pt')
    labels=torch.tensor(labels)
    return (binaries["input_ids"].flatten(),binaries["attention_mask"].flatten(),labels)


In [ ]:
del Malware_Dataset

In [ ]:
del train_dataloader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

#define a batch size
batch_size = 8
train_dataset=Malware_Dataset(tokenizer=tokenizer,df_binaries=train_text,df_labels=train_labels)
# wrap tensors
#train_data = TensorDataset(tokens_train, train_y)

# sampler for sampling the data during training

# dataLoader for train set
train_dataloader = DataLoader(train_dataset, batch_size=batch_size)

# wrap tensors
val_dataset=Malware_Dataset(tokenizer=tokenizer,df_binaries=temp_text,df_labels=temp_labels)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_dataset)

# dataLoader for validation set
val_dataloader = DataLoader(val_dataset,batch_size=batch_size)

check dataloader

In [ ]:
data=next(iter(train_dataloader))
print(data[1].shape)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2277: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


torch.Size([16, 1, 400])


In [ ]:
from transformers import DistilBertModel

In [ ]:
class Bert_arch(nn.Module):

  def __init__(self):
    super(Bert_arch, self).__init__()
    self.bert = AlbertModel.from_pretrained("albert-base-v2")
    self.drop = nn.Dropout(0.3)
    self.out = nn.Linear(768, 7)
  
  def forward(self, input):
    output = self.bert(
        **input
    )
    #print(output[2].shape)
    output = self.drop(output.pooler_output)
    return self.out(output)

In [ ]:
del model

In [ ]:
model = Bert_arch()

# push the model to GPU
model = model.to(device)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.bias', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.LayerNorm.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.bias', 'predictions.dense.weight']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
!nvidia-smi

Fri Apr  8 08:33:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   1273MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from transformers import get_linear_schedule_with_warmup,AdamW

In [ ]:
EPOCHS = 20

optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
total_steps = len(train_dataloader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
  optimizer,
  num_warmup_steps=0,
  num_training_steps=total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
del loss_fn
del optimizer

In [ ]:
model.to(device)

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [ ]:
def train_epoch(
  model, 
  data_loader, 
  loss_fn, 
  optimizer, 
  device, 
  scheduler, 
  n_examples
):
  model = model.train()

  losses = []
  correct_predictions = 0
  
  for x,y,z in data_loader:
    x=x.to(device)

    y=y.to(device)
    z=z.to(device)
    x.squeeze(1)
    y.squeeze(1)
    outputs = model(
      {"input_ids":x,"attention_mask":y}
    )

    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, z)

    correct_predictions += torch.sum(preds == z)
    losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    optimizer.zero_grad()

  return correct_predictions.double() / n_examples, np.mean(losses)

In [ ]:
data=next(iter(train_dataloader))
print(data[1].shape)

torch.Size([16, 400])


In [ ]:
def evaluate():
  
  print("\nEvaluating...")
  
  # deactivate dropout layers
  model.eval()

  total_loss, total_accuracy = 0, 0
  
  # empty list to save the model predictions
  total_preds = []

  # iterate over batches
  for step,batch in enumerate(val_dataloader):
    
    # Progress update every 50 batches.
    if step % 50 == 0 and not step == 0:
      
      # Calculate elapsed time in minutes.
      elapsed = format_time(time.time() - t0)
            
      # Report progress.
      print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

    # push the batch to gpu
    batch = [t.to(device) for t in batch]

    sent_id, mask, labels = batch

    # deactivate autograd
    with torch.no_grad():
      
      # model predictions
      preds = model(sent_id, mask)

      # compute the validation loss between actual and predicted values
      loss = loss_fn(preds,labels)

      total_loss = total_loss + loss.item()

      preds = preds.detach().cpu().numpy()

      total_preds.append(preds)

  # compute the validation loss of the epoch
  avg_loss = total_loss / len(val_dataloader) 

  # reshape the predictions in form of (number of samples, no. of classes)
  total_preds  = np.concatenate(total_preds, axis=0)

  return avg_loss, total_preds


In [ ]:
data=next(iter(train_dataloader))
print(data[0].shape)

KeyError: ignored

In [ ]:
%%time
from collections import defaultdict
history = defaultdict(list)
best_accuracy = 0
EPOCHS=20
for epoch in range(EPOCHS):

  print(f'Epoch {epoch + 1}/{EPOCHS}')
  print('-' * 10)
  train_loss = train_epoch(
    model,
    train_dataloader,    
    loss_fn, 
    optimizer, 
    device, 
    scheduler, 
    len(train_text)
  )

  print(f'Train loss {train_loss}')

  history['train_loss'].append(train_loss)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Epoch 1/20
----------


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Train loss (tensor(0.3716, device='cuda:0', dtype=torch.float64), 1.4787056321116883)
Epoch 2/20
----------
Train loss (tensor(0.7224, device='cuda:0', dtype=torch.float64), 0.807673162738536)
Epoch 3/20
----------
Train loss (tensor(0.8285, device='cuda:0', dtype=torch.float64), 0.5931260742624767)
Epoch 4/20
----------
Train loss (tensor(0.8526, device='cuda:0', dtype=torch.float64), 0.5266817577937655)
Epoch 5/20
----------
Train loss (tensor(0.8812, device='cuda:0', dtype=torch.float64), 0.4406168318839879)
Epoch 6/20
----------
Train loss (tensor(0.8962, device='cuda:0', dtype=torch.float64), 0.4180993158449318)
Epoch 7/20
----------
Train loss (tensor(0.9043, device='cuda:0', dtype=torch.float64), 0.37242533119764787)
Epoch 8/20
----------
Train loss (tensor(0.9176, device='cuda:0', dtype=torch.float64), 0.33197955301904125)
Epoch 9/20
----------
Train loss (tensor(0.9250, device='cuda:0', dtype=torch.float64), 0.28631678383852865)
Epoch 10/20
----------
Train loss (tensor(0.9267

In [ ]:
from transformers import AlbertTokenizer, AlbertModel
model = AlbertModel.from_pretrained("albert-base-v2")
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertModel: ['predictions.LayerNorm.bias', 'predictions.dense.bias', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'predictions.decoder.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
output.pooler_output.shape

torch.Size([1, 768])

In [ ]:
torch.save(model, 'albert_with_20_epochs_400_tokens.pth')

In [ ]:
torch.save(model.state_dict(),"abert_with_20_epochs_weight_400_tokens.pth")

In [ ]:
complete_dataset=Malware_Dataset(tokenizer=tokenizer,df_binaries=df["binaries"],df_labels=df["label"])

In [ ]:
complete_dataloader=DataLoader(complete_dataset,batch_size=1)

In [ ]:
new_model=model.bert

In [ ]:
embeddings=[]
targets=[]
new_model.eval()
count=1
for x,y,z in complete_dataloader:
  x=x.to(device)
  y=y.to(device)
  outputs=new_model(**{"input_ids":x,"attention_mask":y})
  print(len(outputs[1]))
  break
  print(outputs[1].shape)
  print(count)
  embeddings.append(outputs[1].flatten().cpu().detach().numpy())
  targets.append(z.numpy())
  count+=1

1


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
torch.cuda.empty_cache()

In [ ]:
!nvidia-smi

Thu Mar 17 21:04:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P0    27W /  70W |   2836MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
embeddings=np.array(embeddings)

In [ ]:
targets=np.array(targets)

In [ ]:
np.save("bert_10_epochs_embeddings.npy",embeddings)

In [ ]:
np.save("bert_10_epochs_targets.npy",targets)

In [ ]:
!zip bert_model_400_tokens.zip bert_with_10_epochs_400_tokens.pth

  adding: bert_with_10_epochs_400_tokens.pth (deflated 7%)


In [ ]:
!zip bert_model_400_tokens_weights.zip bert_with_10_epochs_weight_400_tokens.pth

  adding: bert_with_10_epochs_weight_400_tokens.pth (deflated 7%)


In [ ]:
del new_model

In [ ]:
model.bert.save_pretrained("albert_with_20_epochs_400_tokens.pth")

In [ ]:
new_model=AlbertModel.from_pretrained("albert_with_20_epochs_400_tokens.pth",output_hidden_states=True)

In [ ]:
new_model.to(device)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [ ]:
embeddings=[]
targets=[]
new_model.eval()
count=1
for x,y,z in complete_dataloader:
  x=x.to(device)
  y=y.to(device)
  outputs=new_model(**{"input_ids":x,"attention_mask":y})[2]
  outputs=torch.stack(outputs,dim=0)
  outputs=torch.squeeze(outputs,dim=1)
  outputs=outputs.permute(1,0,2)
  token_vecs_sum = []
  for outputs in outputs:
      #but preferrable is
      sum_vec=torch.sum(outputs[-4:],dim=0)
        # Use `sum_vec` to represent `token`.
      token_vecs_sum.append(sum_vec)
  h=0
  for i in  range(len(token_vecs_sum)):
      h+=token_vecs_sum[i]
  del token_vecs_sum
  print(h.shape)
  embeddings.append(h.tolist())
  print(count)
  #embeddings.append(outputs[1].flatten().cpu().detach().numpy())
  targets.append(z.numpy())
  count+=1

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Streaming output truncated to the last 5000 lines.
torch.Size([768])
3308
torch.Size([768])
3309
torch.Size([768])
3310
torch.Size([768])
3311
torch.Size([768])
3312
torch.Size([768])
3313
torch.Size([768])
3314
torch.Size([768])
3315
torch.Size([768])
3316
torch.Size([768])
3317
torch.Size([768])
3318
torch.Size([768])
3319
torch.Size([768])
3320
torch.Size([768])
3321
torch.Size([768])
3322
torch.Size([768])
3323
torch.Size([768])
3324
torch.Size([768])
3325
torch.Size([768])
3326
torch.Size([768])
3327
torch.Size([768])
3328
torch.Size([768])
3329
torch.Size([768])
3330
torch.Size([768])
3331
torch.Size([768])
3332
torch.Size([768])
3333
torch.Size([768])
3334
torch.Size([768])
3335
torch.Size([768])
3336
torch.Size([768])
3337
torch.Size([768])
3338
torch.Size([768])
3339
torch.Size([768])
3340
torch.Size([768])
3341
torch.Size([768])
3342
torch.Size([768])
3343
torch.Size([768])
3344
torch.Size([768])
3345
torch.Size([768])
3346
torch.Size([768])
3347
torch.Size([768])
3348
torch.

In [ ]:
embeddings=np.array(embeddings)

In [ ]:
targets=np.array(targets)

In [ ]:
np.save("albert_10_epochs_embeddings_last_4.npy",embeddings)
np.save("albert_10_epochs_targets_last_4.npy",targets)

In [ ]:
from IPython.display import FileLink

In [ ]:
FileLink(embeddings)

ValueError: ignored